In [1]:
from pymongo import MongoClient
import pandas as pd
import os
import pathlib
from dotenv import load_dotenv
import datetime
from fredapi import Fred

env_path = pathlib.Path('..') / '.env'
load_dotenv(dotenv_path=env_path)
FRED_API_KEY = os.getenv("FRED_API_KEY")
fred = Fred(api_key=FRED_API_KEY)


client = MongoClient('localhost', 27017)
db = client.project2_db
collection = db.project2_collection

# Define your series dictionary, start and end dates as before
series_dict = {
    'bonds2tr': ('DGS2', 'D'),
    'gdpworld': ('NYGDPMKTPCDWLD', 'A'),
    'gdp': ('GDP', 'Q'),
    'recession': ('JHDUSRGDPBR', 'Q'),
    'bonds10tr': ('DGS10', 'D'),
    'ppi': ('PPIACO', 'M'),
    'cpi': ('CPIAUCSL', 'M'),
    'unrate': ('UNRATE', 'M'),
    'debt': ('GFDEBTN', 'Q'),
    'fedrate': ('FEDFUNDS', 'M'),
    'm0': ('BOGMBASE', 'M'),
    'm3': ('MABMM301USM189S', 'M'),
    'cbasstogdp': ('DDDI06USA156NWDB', 'A'),
    'resins': ('TOTRESNS', 'M'),
    'oil': ('WTISPLC', 'M'),
    'indpro': ('INDPRO', 'M'),
    'houses': ('MSPUS', 'Q'),
    'wages': ('CES3000000008', 'M'),
    'cp': ('CP', 'Q')
}

# Define dates
start_date = datetime.datetime(1970, 1, 1)
end_date = datetime.datetime(2022, 12, 31)

# Initialize DataFrames
dfs = {'D': pd.DataFrame(), 'M': pd.DataFrame(), 'Q': pd.DataFrame(), 'A': pd.DataFrame()}


def fetch_data(df, var_name, series_id):
    series_data = fred.get_series(series_id)
    series_df = series_data.to_frame(name=var_name)
    series_df.index.name = 'date'
    return df.join(series_df, how='outer') if not df.empty else series_df


for var_name, (series_id, freq) in series_dict.items():
    dfs[freq] = fetch_data(dfs[freq], var_name, series_id)

In [2]:
df = {}
df['D_M'] = dfs['D'].resample('M').last()
df['M_M'] = dfs['M'].resample('M').last()
df['Q_M'] = dfs['Q'].resample('M').ffill()
new_dates = pd.date_range(
    start=df['Q_M'].index.min(), end='2022-12-31', freq='M')
df['Q_M'] = df['Q_M'].reindex(new_dates).ffill()
df['A_M'] = dfs['A'].resample('M').ffill()

In [5]:
data = pd.concat([df['D_M'], df['M_M'], df['Q_M'], df['A_M']], axis=1)

In [9]:
data['wages_month'] = round(data['wages'] * 168, 2)
data['house_wages'] = round(data['houses']/data['wages_month'], 2)
data['iyc'] = round(data['bonds10tr'] - data['bonds2tr'], 2)
data['gdp_pct'] = data['gdp'].pct_change(periods=4) * 100
data['gdp_pct_ma4'] = data['gdp_pct'].rolling(window=4).mean()
data['gdpworld_pct'] = data['gdpworld'].pct_change(12) * 100
data['debt_to_gdp'] = round(data['debt'] / data['gdp']/1000, 2)
data['m0_to_gdp'] = round(data['m0'] / data['gdp']/1000 * 100, 2)
data['m3_to_gdp'] = round((data['m3']/1000000) / data['gdp']/1000 * 100, 2)
data['cpi_pct'] = data['cpi'].pct_change(periods=12) * 100
data['ppi_pct'] = data['ppi'].pct_change(periods=12) * 100
# data = data[start_date:end_date]
data
# data[['gdp_pct', 'gdp_pct_ma4', 'gdpworld', 'gdpworld_pct']].tail(20)

,date,bonds2tr,bonds10tr,ppi,cpi,unrate,fedrate,m0,m3,resins,...,house_wages,iyc,gdp_pct,gdp_pct_ma4,gdpworld_pct,debt_to_gdp,m0_to_gdp,m3_to_gdp,cpi_pct,ppi_pct
0,1970-01-31,NaN,7.75,36.500,37.900,3.9,8.98,76400.0,5.896000e+11,28.9,...,45.02,NaN,NaN,NaN,NaN,0.35,7.27,56.09,NaN,NaN
1,1970-02-28,NaN,6.90,36.700,38.100,4.2,8.98,75200.0,5.863000e+11,27.9,...,44.88,NaN,NaN,NaN,NaN,0.35,7.15,55.77,NaN,NaN
2,1970-03-31,NaN,7.08,36.700,38.300,4.4,7.76,75300.0,5.873000e+11,27.5,...,44.60,NaN,NaN,NaN,NaN,0.35,7.16,55.87,NaN,NaN
3,1970-04-30,NaN,7.82,36.800,38.500,4.6,8.10,76300.0,5.884000e+11,28.1,...,45.53,NaN,NaN,NaN,NaN,0.35,7.15,55.13,NaN,NaN
4,1970-05-31,NaN,7.95,36.800,38.600,4.8,7.95,76600.0,5.915000e+11,27.9,...,45.10,NaN,1.538718,NaN,NaN,0.35,7.18,55.42,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631,2022-08-31,3.45,3.15,269.546,295.320,3.7,2.33,5582200.0,2.165990e+13,3305.9,...,110.76,-0.30,1.763080,2.434285,3.798021,1.19,21.47,83.32,8.227361,15.479297
632,2022-09-30,4.22,3.83,267.898,296.539,3.5,2.56,5410900.0,2.152520e+13,3131.4,...,110.28,-0.39,1.763080,2.360498,3.798021,1.19,20.82,82.81,8.214854,13.671195
633,2022-10-31,4.51,4.10,265.061,297.987,3.7,3.08,5339600.0,2.143270e+13,3055.7,...,112.46,-0.41,3.382880,2.691660,3.798021,1.19,20.22,81.16,7.762493,10.228516
634,2022-11-30,4.38,3.68,263.157,298.598,3.6,3.78,5418700.0,2.139870e+13,3126.2,...,111.49,-0.70,1.591736,2.125194,3.798021,1.19,20.52,81.03,7.135348,8.167309


In [7]:
data.index.name = 'date'
data = data.reset_index()

data_to_insert = data.to_dict(orient='records')
collection.insert_many(data_to_insert)

In [6]:
# data.to_csv('data.csv')
# data['date'] = pd.to_datetime(data['date'])
# data = data.set_index('date', inplace=True)
# data.index.name = 'date'
# data


In [17]:
# quaterly_last_record = data.resample('Q').last().dropna(how='all')
# quaterly_last_record.to_csv('quaterly_last_record.csv')


In [190]:
# quaterly_last_record